In this I'm going to test BERTopic for topic modeling. It seems to have significant advantages over LDA.

Going to just follow a tutorial and use their dataset for now

In [2]:
from bertopic import BERTopic
import pandas as pd
import pickle

c:\Users\rharter2\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# Since BERT uses sentence embeddings in it's process we want the stop words but the removed ones are useless
# Testing on this one but it's really small so idk if the topics will be good
dfC = pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\opiates_comments1.json")
display(dfC.shape)
dfC = dfC.loc[(dfC['body'] != '') & (dfC['body'] != '[deleted]') & (dfC['body'] != '[removed]')]
dfC.reset_index(drop=True, inplace=True)

data = dfC['body'].to_list()
display(len(data))

(1104, 23)

871

In [20]:
# precalculate sentence embeddings so they don't have to be done each time and save them
from sentence_transformers import SentenceTransformer
if 1:
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # fast but pretty accurate embeddings
    embeddings = embedding_model.encode(data, show_progress_bar=True)
    
    with open("embeddings.pkl", "wb") as fOut:
        pickle.dump({'embeddings': embeddings},fOut)



Batches: 100%|██████████| 28/28 [00:14<00:00,  1.98it/s]


In [19]:
# Load embeddings from file
if 0:
    with open("embeddings.pkl", "rb") as fIn:
        cache_data = pickle.load(fIn)
        embeddings = cache_data['embeddings']

In [ ]:
# Default representations. Want to try another representation that has less stopwords
topicModel = BERTopic(verbose=True, embedding_model="all-MiniLM-L6-v2") 
topics, probs = topicModel.fit_transform(data, embeddings)

In [13]:
# A maybe better representation?
from bertopic.representation import KeyBERTInspired

topicModel = BERTopic(verbose=True, embedding_model="all-MiniLM-L6-v2", representation_model=KeyBERTInspired()) 
topics, probs = topicModel.fit_transform(data, embeddings)

2023-10-20 17:41:11,386 - BERTopic - Reduced dimensionality
2023-10-20 17:41:11,433 - BERTopic - Clustered reduced embeddings


In [14]:
topicModel.save("keyBertModelReddit", serialization='safetensors')

In [15]:
topicModel.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,870,0_addiction_opiates_taking_feel,"[addiction, opiates, taking, feel, take, get, ...",[When it comes to narcotics doctors and pharma...
1,1,234,1_deleted___,"[deleted, , , , , , , , , ]","[[deleted], [deleted], [deleted]]"


In [16]:
topicModel.get_topic(1)

[('deleted', 0.57557726),
 ('', 0.3403504),
 ('', 0.3403504),
 ('', 0.3403504),
 ('', 0.3403504),
 ('', 0.3403504),
 ('', 0.3403504),
 ('', 0.3403504),
 ('', 0.3403504),
 ('', 0.3403504)]

In [ ]:
topicModel.visualize_topics()